In [1]:
import cntk as C
from deep_model import *
#import deep_model
prefix = "/home/t-haohu/IndustryClassifier/Data"
suffix = "180days_sample"

data_sample_body_ctf_train = "{}/ready/train_{}_body.ctf".format(prefix,suffix)
data_sample_body_ctf_test = "{}/ready/test_{}_body.ctf".format(prefix,suffix)
data_sample_ctf_train = "{}/ready/train_{}.ctf".format(prefix,suffix)
data_sample_ctf_test = "{}/ready/test_{}.ctf".format(prefix,suffix)

data_title_dict = "{}/ready/title_{}.pkl".format(prefix,suffix)
data_body_dict = "{}/ready/body_{}.pkl".format(prefix,suffix)
data_industry_sample = "{}/ready/industry_{}.wl".format(prefix,suffix)


init_param_setting(vocabulary= 23184,industry = 19,title=23184,body = 60491)

file_param_setting(data_industry_sample,data_title_dict,data_body_dict)
super_param_setting(embed=300,hidden=200,sentence=30,doc=100,minibatch=8192,max_epoch_size = 10,l2_reg =0,lr = [5e-4*8192]*2+[1e-4*8192])
#models = ['cnn']
#for m in models:
#    do_train(m,data_sample_ctf_train,data_sample_ctf_test,suffix,42000)
#super_param_setting(embed=300,hidden=200,sentence=50,minibatch=2048,max_epoch_size = 30,l2_reg =0)
models = ['cnn_body']
for m in models:
    do_train(m,data_sample_body_ctf_train,data_sample_body_ctf_test,suffix,42000)

cnn_body
[Output('hidden', [#], [19]), Output('drop1', [#], [300 x 1 x 1]), Output('Splice125132_Output_0', [#], [300 x 1 x 1]), Output('pooling', [#], [50 x 1 x 1]), Output('Block76523_Output_0', [#], [50 x 29 x 1]), Output('Block36539_Output_0', [#], [30 x 300]), Output('Block36539_Output_1', [#], [30 x 1]), Output('embed_title', [#, 11], [300]), Output('pooling', [#], [50 x 1 x 1]), Output('Block74532_Output_0', [#], [50 x 30 x 1]), Output('pooling', [#], [50 x 1 x 1]), Output('Block78514_Output_0', [#], [50 x 28 x 1]), Output('pooling', [#], [50 x 1 x 1]), Output('Block86976_Output_0', [#], [50 x 99 x 1]), Output('Block64871_Output_0', [#], [100 x 300]), Output('Block64871_Output_1', [#], [100 x 1]), Output('embed_body', [#, 21], [300]), Output('pooling', [#], [50 x 1 x 1]), Output('Block80505_Output_0', [#], [50 x 100 x 1]), Output('pooling', [#], [50 x 1 x 1]), Output('Block93447_Output_0', [#], [50 x 98 x 1])]
Training 25288519 parameters in 16 parameter tensors.
Learning rate p

In [ ]:
from cntk import load_model
evaluate(test_file,model_func,is_body=True):